In [1]:
# 如果导入需求包出现问题，运行这个模块。（需要等待一段时间）
! pip3 install openpyxl pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/


In [2]:
# 导入需求包
import openpyxl
import csv
import os
import time
from openpyxl import Workbook
import pandas as pd

In [3]:
# 读取原始数据
wb = openpyxl.load_workbook(os.getcwd()+r"\haiyao_v3.xlsx")
ws = wb.active

In [4]:
def gen_item(raw_ws):
    """
    生成实体信息csv
    """
    types = []
    for type_cell in raw_ws[1]:
        type_ = type_cell.value
        types.append([type_, []])
        
    for line in raw_ws[2:raw_ws.max_row]:
        for i, cell in enumerate(line):
            values = cell.value
            if not values:
                continue
            values = str(values)
            values = values.replace("\n", "")
            values = values.replace(" ", "")
            values = values.replace("，", ",")
            values = values.replace("、", ",")
            values = values.replace(".", ",")
            values = values.replace("。", ",")
            values = values.replace("：", ",")
            values = values.replace(":", ",")
            values = values.strip()
            for value in values.split(','):
                if not value:
                    continue
                if value not in types[i][1]:
                    types[i][1].append(value)
                
    time_ = time.strftime("%m%d_%H%M%S", time.localtime()) 
    folder = str(time_)+'_items'
    for type_msg in types:
        type_ = type_msg[0]
        items = type_msg[1]
        df = pd.DataFrame({type_: items}) 
        print(df)
        if not os.path.exists(folder):
            os.makedirs(folder)
        df.to_csv(folder+'/'+type_+'_items.csv',encoding='utf_8_sig', index=False) 

In [5]:
# 生成实体信息csv
gen_item(ws)

      编号
0      1
1      2
2      3
3      4
4      5
..   ...
126  127
127  128
128  129
129  130
130  131

[131 rows x 1 columns]
       药物
0      玉屑
1      车渠
2     金线矾
3    波斯白矾
4      金屑
..    ...
126   文林郎
127   无漏子
128   摩厨子
129   君迁子
130    师草

[131 rows x 1 columns]
    部
0  玉石
1   草
2   木
3   兽
4  虫鱼
5  果米
       产地
0      昆仑
1      钟山
2      西国
3     波斯国
4     大秦国
5     大食国
6    蜀中雅州
7      南海
8     新罗国
9      东海
10     岭南
11     西海
12     安南
13     黔中
14     奚国
15     秦国
16   广南山谷
17     西戎
18     剽地
19     广南
20     源州
21     西域
22     海南
23     鄂州
24     岳州
25   南中八郡
26    新平县
27  岭南及诸海
28  西海波斯国
29   山东海水
30     广州
31     云南
32     交趾
33    卑占国
   味
0  咸
1  酸
2  涩
3  甘
4  苦
5  辛
    性
0   寒
1  大寒
2   温
3   平
4  微温
5  温平
6  小温
7   冷
8  大温
9  微寒
         毒性
0         无
1         有
2  生者有毒熟者无毒
3       有小毒
4       有大毒
    非药用功效
0    鬼气注忤
1    风邪鬼注
2      辟蛀
3      染发
4     辟蚊蚋
5     益颜色
6    鬼魅邪精
7   辟邪恶之气
8      辟恶
9      轻身
10     延年
11     长生
12   鬼疰心气
13     辟谷
14   心气鬼疰

In [6]:
def gen_relation(raw_ws, split=True):
    """
    生成关系信息csv
    split设置为False时，将合并相同表头的文件。
    """
    relation_names = []
    
    drugs = []
    relations = []
    params = []
    
    tables = {}
    
    
    
    for cell in raw_ws[1]:
        relation_names.append(cell.value)
        tables[cell.value] = {
            "drugs": [],
            "relations": [],
            "params": [] 
        }
    print(relation_names)
    
    tables["从属关系"] = {
        "first_layer_params": [],
        "relations": [],
        "second_layer_params": [] 
    }
    relationship_deduplication = set()
        
    for line in raw_ws[2:raw_ws.max_row]:
        drug = line[1].value
        if not drug:
            break
        drug = drug.strip()
        for i, cell in enumerate(line):
            if i == 1:
                continue
                
            values = cell.value
            if not values:
                continue
            values = str(values)
            values = values.replace("\n", "")
            values = values.replace(" ", "")
            values = values.replace("，", ",")
            values = values.replace("、", ",")
            values = values.replace(".", ",")
            values = values.replace("。", ",")
            values = values.replace("：", ":")
            values = values.strip()
            for value in values.split(','):
                relation = relation_names[i]
                if not value:
                    continue
                elif ":" in value:
                    v1, v2 = value.split(":")
                    tables[relation]["drugs"].append(drug)
                    tables[relation]["relations"].append(relation_names[i]+"是")
                    tables[relation]["params"].append(v1)
                    tables[relation]["drugs"].append(drug)
                    tables[relation]["relations"].append(relation_names[i]+"具体为")
                    tables[relation]["params"].append(v2)
                    tmp_key_to_deduplicate = v1+relation_names[i]+v2
                    if tmp_key_to_deduplicate not in relationship_deduplication:
                        relationship_deduplication.add(tmp_key_to_deduplicate)
                        tables["从属关系"]["first_layer_params"].append(v1)
                        tables["从属关系"]["relations"].append(relation_names[i]+"-次级属性有")
                        tables["从属关系"]["second_layer_params"].append(v2)
                else:
                    tables[relation]["drugs"].append(drug)
                    tables[relation]["relations"].append(relation_names[i]+"是")
                    tables[relation]["params"].append(value)
                
    folder = str(time.strftime("%m%d_%H%M%S", time.localtime()))+"_relations"
    df = pd.DataFrame({"药名": [], "关系": [], "属性": []}) 
    for relation in relation_names:
        if relation == "药物":
            continue
        table = tables[relation]
        df_t = pd.DataFrame({"药名": table["drugs"], 
                           "关系": table["relations"],
                           "属性": table["params"]}) 
        print(df_t)
        df = pd.concat([df,df_t],axis=0,ignore_index=True)

    if not os.path.exists(folder):
        os.makedirs(folder)
    df.to_csv(folder+'/药物属性_relations.csv',encoding='utf_8_sig', index=False) 
        
        
    table = tables["从属关系"]
    df = pd.DataFrame({"上级属性": table["first_layer_params"], 
                       "关系": table["relations"],
                       "次级属性": table["second_layer_params"]}) 
    print(df)
    if not os.path.exists(folder):
        os.makedirs(folder)
    df.to_csv(folder+'/从属_relations.csv',encoding='utf_8_sig', index=False)

In [7]:
# 生成关系信息csv
gen_relation(ws)

['编号', '药物', '部', '产地', '味', '性', '毒性', '非药用功效', '内科功效', '内科主治', '外科主治', '妇科主治', '儿科主治']
       药名   关系   属性
0      玉屑  编号是    1
1      车渠  编号是    2
2     金线矾  编号是    3
3    波斯白矾  编号是    4
4      金屑  编号是    5
..    ...  ...  ...
126   文林郎  编号是  127
127   无漏子  编号是  128
128   摩厨子  编号是  129
129   君迁子  编号是  130
130    师草  编号是  131

[131 rows x 3 columns]
       药名  关系  属性
0      玉屑  部是  玉石
1      车渠  部是  玉石
2     金线矾  部是  玉石
3    波斯白矾  部是  玉石
4      金屑  部是  玉石
..    ...  ..  ..
126   文林郎  部是  果米
127   无漏子  部是  果米
128   摩厨子  部是  果米
129   君迁子  部是  果米
130    师草  部是  果米

[131 rows x 3 columns]
       药名   关系   属性
0      玉屑  产地是   昆仑
1      玉屑  产地是   钟山
2      车渠  产地是   西国
3     金线矾  产地是  波斯国
4    波斯白矾  产地是  大秦国
..    ...  ...  ...
108    橄榄  产地是   南海
109   海松子  产地是   云南
110   偏桃仁  产地是  卑占国
111   都角子  产地是   广南
112   摩厨子  产地是   西域

[113 rows x 3 columns]
      药名  关系 属性
0     玉屑  味是  咸
1    金线矾  味是  咸
2    金线矾  味是  酸
3    金线矾  味是  涩
4   波斯白矾  味是  酸
..   ...  .. ..
59    豆蔻  味是  辛
60    荔枝  味是  甘